In [1]:
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge pyspark

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==2019.10=py37_0
  - defaults/noarch::dask==2.5.2=py_0
  - defaults/linux-64::statsmodels==0.10.1=py37hdd07704_0
  - conda-forge/noarch::pyspark==2.4.5=py_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::seaborn==0.9.0=py37_0
done

## Package Plan ##

  environment location: /home/bas/anaconda3

  added / updated specs:
    - pyspark


The following NEW packages will be INSTALLED:

  _anaconda_depends  pkgs/main/linux-64::_anaconda_depends-2019.03-py37_0
  pandas             conda-forge/linux-64::pandas-1.0.3-py37h0da4684_0
  python_abi         conda-forge/linux-64::python_abi-3.7-1_cp37m

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificates-2019.8.28-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0
  certifi               pkgs/main::certifi-2

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, size

In [3]:
spark = SparkSession.builder.appName("Packt").getOrCreate()

In [4]:
data = spark.read.csv('../../Datasets/netflix_titles_nov_2019.csv', header='true')
data.show()

+--------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+---------+--------------------+--------------------+-------+
| show_id|               title|            director|                cast|             country|       date_added|release_year|rating| duration|           listed_in|         description|   type|
+--------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+---------+--------------------+--------------------+-------+
|81193313|           Chocolate|                null|Ha Ji-won, Yoon K...|         South Korea|November 30, 2019|        2019| TV-14| 1 Season|International TV ...|Brought together ...|TV Show|
|81197050|Guatemala: Heart ...|Luis Ara, Ignacio...|   Christian Morales|                null|November 30, 2019|        2019|  TV-G|   67 min|Documentaries, In...|From Sierra de la...|  Movie|
|81213894|     The Zoya Factor|    

In [5]:
movies = data.filter((col('type') == 'TV Show') & ((col('rating') == 'TV-G') | (col('rating') == 'TV-Y')))
movies.show()

+--------+--------------------+--------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+-------+
| show_id|               title|director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|   type|
+--------+--------------------+--------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+-------+
|80213643|     Chip and Potato|    null|Abigail Oliver, A...|Canada, United Ki...|              null|        2019|  TV-Y|2 Seasons|            Kids' TV|Lovable pug Chip ...|TV Show|
|80117560|Trolls: The Beat ...|    null|Amanda Leighton, ...|       United States|              null|        2019|  TV-G|8 Seasons|Kids' TV, TV Come...|As Queen Poppy we...|TV Show|
|80115338|         Llama Llama|    null|Jennifer Garner, ...|       United States|        

In [6]:
transformed = movies.withColumn('count_lists', size(split(movies['listed_in'], ',')))

In [7]:
selected = transformed.select('title', 'cast', 'rating', 'release_year', 'duration', 'count_lists', 'listed_in', 'description')
selected.show()

+--------------------+--------------------+------+------------+---------+-----------+--------------------+--------------------+
|               title|                cast|rating|release_year| duration|count_lists|           listed_in|         description|
+--------------------+--------------------+------+------------+---------+-----------+--------------------+--------------------+
|     Chip and Potato|Abigail Oliver, A...|  TV-Y|        2019|2 Seasons|          1|            Kids' TV|Lovable pug Chip ...|
|Trolls: The Beat ...|Amanda Leighton, ...|  TV-G|        2019|8 Seasons|          2|Kids' TV, TV Come...|As Queen Poppy we...|
|         Llama Llama|Jennifer Garner, ...|  TV-Y|        2019|2 Seasons|          1|            Kids' TV|Beloved children'...|
|Bella and the Bul...|Brec Bassinger, C...|  TV-G|        2015| 1 Season|          2|Kids' TV, TV Come...|The life of cheer...|
|          Victorious|Victoria Justice,...|  TV-G|        2013|3 Seasons|          2|Kids' TV, TV Come..

In [9]:
selected.write.csv('transformed2', header='true')

In [11]:
# note: the filename ('part-....') will differ in your local machine
!head transformed2/part-00000-c6aba32b-de71-431d-972c-ab14f124d5c2-c000.csv

title,cast,rating,release_year,duration,count_lists,listed_in,description
Chip and Potato,"Abigail Oliver, Andrea Libman, Briana Buckmaster, Brian Dobson, Chance Hurstfield, Dominic Good, Emma Jayne Maas, Evan Byarushengo, Scotia Anderson, Alessandro Juliani",TV-Y,2019,2 Seasons,1,Kids' TV,"Lovable pug Chip starts kindergarten, makes new friends and tries new things – with a little help from Potato, her secret mouse pal."
Trolls: The Beat Goes On!,"Amanda Leighton, Skylar Astin, Ron Funches, David Fynn, David Koechner, David Kaye, Sean T. Krishnan, Sam Lerner, Patrick Pinney, Kevin Michael Richardson, Kari Wahlgren, Fryda Wolff",TV-G,2019,8 Seasons,2,"Kids' TV, TV Comedies","As Queen Poppy welcomes a new time of peace in Troll Village with parties, sports and holiday celebrations, Branch tries to be more fun-loving."
Llama Llama,"Jennifer Garner, Shayle Simons, Vania Gill, Islie Hirvonen, Brendon Sunderland, Austin Abell, Evans Johnson, Kathleen Barr, David Hoole",TV-Y,2019,2 Seasons,1